In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import tensorflow as tf
from tensorflow import keras

house_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train_labels = house_data["SalePrice"]
train_data = house_data
train_data.drop(columns=["SalePrice"])




,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


Columns contain mixture of categorical data and numerical data, these will require different preprocessing. We have:
* Numerical data: This needs to be scaled between 0 and 1.
* Categorical data (including the zoning code!): This needs to be one-hot encoded.
* 'Quality' categories: This needs to be converted to an equal numeric range between 0 and 1.

Missing values must also be managed:
* For numerical columns, exclude all NA rows to find the mean, then replace NA with the mean wherever it appears
* For categorical columns, NA will be included as a category
* Neither the training data nor test data contains any nulls (but if they were there then they should be replaced with NA first, then subjected to the same processing.

All these steps must be performed on the training data and validation data separately after splitting them! (wrap in function to take each) otherwise replacement with the mean of the whole dataset would lead to information leakage between train and validation sets.

In [3]:
lotshapeencode = {"Reg":0, "IR1":1, "IR2":2, "IR3":3}
train_data["LotShape"] = train_data["LotShape"].replace(lotshapeencode)
landslopeencode = {"Gtl":0, "Mod":1, "Sev":2}
train_data["LandSlope"] = train_data["LandSlope"].replace(landslopeencode)
qualcondencode = {"Ex":0, "Gd":1, "TA":2, "Fa":3, "Po":4}
train_data["ExterQual"] = train_data["ExterQual"].replace(qualcondencode)
train_data["ExterCond"] = train_data["ExterCond"].replace(qualcondencode)
train_data["BsmtQual"] = train_data["BsmtQual"].replace(qualcondencode)
train_data["BsmtCond"] = train_data["BsmtCond"].replace(qualcondencode)
exposureencode = {"Gd":0, "Av":1, "Mn":2, "No":3}
train_data["BsmtExposure"] = train_data["BsmtExposure"].replace(exposureencode)
train_data["HeatingQC"] = train_data["HeatingQC"].replace(qualcondencode)
train_data["KitchenQual"] = train_data["KitchenQual"].replace(qualcondencode)
functionalencode = {"Typ":0, "Min1":1, "Min2":2, "Mod":3, "Maj1":4, "Maj2":5, "Sev":6, "Sal":7}
train_data["Functional"] = train_data["Functional"].replace(functionalencode)
train_data["FireplaceQu"] = train_data["FireplaceQu"].replace(qualcondencode)
finishencode = {"Fin":0, "Rfn":1, "Unf":2}
train_data["GarageFinish"] = train_data["GarageFinish"].replace(finishencode)
